# Escopo de variáveis

***DISCLAIMER: Este notebook foi escrito com base no que li [neste](https://docs.julialang.org/en/v1/manual/variables-and-scoping/) capítulo do manual***.


O escopo de uma variável permite-nos criar diferentes variáveis com o mesmo nome em partes diferentes do código. Normalmente ouvimos falar do *escopo global* onde as variáveis são declaradas fora de funções, loops e structs. E o *escopo local* onde as variáveis são criadas dentro de funções, structs ou loops.

Os *escopos locais* ainda se subdividem em: *soft* e *hard*.

Deixo [aqui](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#man-scope-table) a lista de construtores que criam estes 3 tipos de escopos.

***Nota*** &#8595;

<hr>

Os blocos `if` e `begin` não criam escopos, eles mantêm as variáveis criadas dentro deles, acessíveis fora deles. Vejamos uma exemplo entre `if` e o `for` (este já cria um escopo local).

<hr>

In [3]:
x = 1

if x == 1
    # Vamos criar uma nova variável
    y =3
end
# Vamos tentar aceder à nova variável (vai funcionar)
y

3

In [5]:
a = 4

for i in 1:a
    # Criar constamente uma nova variável
    b = 1
end

# Vamos tentar aceder à nova variável
# Vai falar que não está definida, isto porque, ela foi
# criada no for e eliminada após o término do mesmo
b

LoadError: UndefVarError: b not defined

O **escopo lexical** que Julia utiliza permite que quando estamos dentro de um escopo local, o programa utilize as variáveis definidas nesse escopo (mesmo que haja variáveis globais definidas com os mesmos nomes).

Além disso variáveis dentro de um escopo interno (como de um `for`) conseguem ver todas as variáveis dentro desse escopo e fora dele. Já as variáveis fora desse escopo interno, não conseguem aceder às variáveis dentro do escopo interno.

In [6]:
module Test
    x = 3
    foo() = x
end;

In [8]:
import .Test

# Variável global
x = -1

# Irá utilizar a variável x utilizada dentro do escopo
# do module
Test.foo()

3

## Escopo Global

Como devem ter visto na tabela os `module` criam variáveis globais (variáveis globais dentro do escopo deles).

Podemos importar `modules` dentro de outros `modules`, porém as variáveis desses `modules` importados são apenas *read-only*.

In [10]:
module A
    mod_a = 45
end;


module B
    import ..A
    
    # Posso ler a variável mod_a de A
    println("Valor de mod_a = ", A.mod_a)

    # Mas não vou poder modificar
    A.mod_a = 12
end;

Valor de mod_a = 45


LoadError: cannot assign variables in other modules

In [12]:
# Verificar o escopo global de um module
module C
    souglobal = "Global"

    function copia()
        souglobal = "Local"
        println("souglobal $souglobal")
    end
    
    # A modificação dentro da função, não foi uma modificação à variável global
    # mas sim a criação de uma variável local dentro da função copia
    println("souglobal $souglobal")
    copia()
end;

souglobal Global
souglobal Local


## Escopo Local

Já vimos no exemplo acima (na função `copia`) que para criar variáveis locais não precisamos de colocar nada antes do nome da variável, o que significa que Julia faz **implicitamente** a declaração da nova variável local.

Porém podemos colocar `local` antes de uma variável local, para fazê-lo explicitamente: `local soulocal = "Local"`

Vamos passar por algumas situações que podem ocorrer:

* [1] Se eu definir uma variável local `x` e ela já existir? Então na realidade eu vou apenas modificar o valor da variável `x` que já existe.

* [2] Se eu definir uma variável `x` que ainda não existe no *hard scope*, ou seja, dentro de um bloco `function`, `macro` etc ... Julia vai criar essa variável local `x` para mim.

* [3] Se eu definir uma variável `x` que ainda não existe no *soft scope*, ou seja, dentro de um bloco `try/catch`, `struct` etc ... Então pode acontecer 2 coisas:
    
    * Se `x` ainda não existir no escopo global, então será criada uma variável `x` local no soft scope.
    * Se `x` já existir no escopo global, então caso estejamos em um modo não interativo, receberemos um warning de ambiguidade, MAS `x` é criado localmente no soft scope. No caso de um ambiente interativo (REPL) iremos modificar a variável `x` global.